# Tanzania water pumps project : Synthesis

In [1]:
import numpy as np
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

## Functions for data cleaning

In [2]:
def set_less_freq(val,most_freq,other):
    """Replaces values that are not in most_freq list by other"""
    if val not in most_freq:
        return other
    else:
        return val

In [3]:
def clean_val(s_in,map_df):
    """This function replaces the values in Serie s_in
     according to map_df.before and map_df.after"""
    s_out = s_in.copy()
    s_out = s_out.replace(to_replace=list(map_df.before), value=list(map_df.after))
    s_out = s_out.str.strip()
    s_out = s_out.str.lower()
    return s_out

In [4]:
def fit_latitude_longitude(df):
    """This function gets regions means"""
    geo_df = df[df.latitude < -0.1]
    geo_df = geo_df[geo_df.longitude > 29]
    geo_df = geo_df[['latitude','longitude','region']]
    mean_df = geo_df.groupby('region')[['latitude','longitude']].mean()
    return mean_df

In [23]:
def clean_latitude_longitude(df, mean_df):
    """This function replaces 0 or near 0 values by the mean of the region"""
    out_df = df.copy()
    for i in df.index:
        if df.loc[i,'latitude'] > -0.1 or df.loc[i,'longitude'] < 29:
            region = df.loc[i,'region']
            out_df.loc[i,'latitude'] = mean_df.loc[region,'latitude']
            out_df.loc[i,'longitude'] = mean_df.loc[region,'longitude']
    return out_df.latitude, out_df.longitude

## Class for data cleaning

In [6]:
class clean_water():
    """This class does all the data cleaning and categorization 
    as done in analysis and machine learning notebooks"""
    
    def __init__(self):
        # most frequent categories to keep
        self.n_to_keep = 100
        
        # col to drop
        self.col_to_drop = ['date_recorded','wpt_name','num_private','subvillage',
                            'region_code','recorded_by','scheme_management',
                            'scheme_name','extraction_type_group','extraction_type',
                            'management_group','payment_type','water_quality',
                            'quantity_group','source','source_class',
                            'waterpoint_type','public_meeting','permit']
        
        self.col_to_cat=['funder','installer','basin','region','district_code','lga',
                         'ward','extraction_type_class','management','source_type',
                         'waterpoint_type_group']
        
        # fix funder misspellings
        self.funder_map = pd.read_csv('../Data/funder map.csv', header=None, 
                                      names=['before','after'],index_col=False)
        
        # fix installer misspellings
        self.installer_map = pd.read_csv('../Data/installer map.csv', header=None, 
                                         names=['before','after'],index_col=False)
    
        # ordered categories for payment
        self.payment_type = pd.CategoricalDtype(categories=['other','never pay',
                                                            'pay when scheme fails',
                                                            'pay annually','pay monthly',
                                                            'pay per bucket'],
                                                ordered=True)
        
        # ordered categories for quality_group
        self.quality_group_type = pd.CategoricalDtype(categories=['unknown','fluoride',
                                                                  'salty','colored',
                                                                  'milky','good'], 
                                                      ordered=True)
        
        # ordered categories for quantity
        self.quantity_type = pd.CategoricalDtype(categories=['unknown','dry','insufficient',
                                                             'seasonal','enough'], 
                                                 ordered=True)
        
    def fit(self, X):
        # funder: fix spelling then get most frequent values
        s_funder = clean_val(X.funder,self.funder_map)
        self.funder_most_freq = list(s_funder.value_counts()[:self.n_to_keep].index)
        
        # installer: fix spelling then get most frequent values
        s_installer = clean_val(X.installer,self.installer_map)
        self.installer_most_freq = list(s_installer.value_counts()[:self.n_to_keep].index)
        
        # latitude, longitude: get mean values for each region
        self.lat_lon_mean = fit_latitude_longitude(X)
        
        # ward: get most frequent values
        self.ward_most_freq = list(X.ward.value_counts()[:self.n_to_keep].index)

        # population: get median value
        self.pop_median = X[X.population !=0].population.median()
        
        # construction_year: get mean value
        self.year_mean = X[X.construction_year !=0].construction_year.mean()
            
        return self
    
    def transform(self, X_in):
        X_out = X_in.copy()
        # funder: fix spelling then replace less frequent values with other
        s_funder = clean_val(X_in.funder, self.funder_map)
        X_out.funder = s_funder.apply(set_less_freq, args=(self.funder_most_freq,'other'))

        # installer: fix spelling then replace less frequent values with other
        s_installer = clean_val(X_in.installer, self.installer_map)
        X_out.installer = s_installer.apply(set_less_freq, args=(self.installer_most_freq,'other'))

        # latitude, longitude: replace 0 or near 0 values with mean of the region
        X_out.latitude, X_out.longitude = clean_latitude_longitude(X_in, self.lat_lon_mean)
        
        # ward: replace less frequent values with other
        X_out.ward = X_in.ward.apply(set_less_freq, args=(self.ward_most_freq,'other'))

        # population: replace 0 values with median
        X_out.population = X_in.population.replace(to_replace=0,value=self.pop_median)
        
        # public_meeting: replace Nan with False
        X_out.public_meeting = X_in.public_meeting.fillna(False)
        
        # permit: replace Nan with False
        X_out.permit = X_in.permit.fillna(False)
        
        # construction_year: replace 0 values with mean
        X_out.construction_year = X_in.construction_year.replace(to_replace=0,value=self.year_mean)
        
        # management: replace unknown with other
        X_out.management = X_in.management.replace(to_replace='unknown',value='other')
        
        # payment: replace unknown with other and categorize
        X_out.payment = X_in.payment.replace(to_replace='unknown', value='other')
        X_out.payment = X_out.payment.astype(self.payment_type).cat.codes

        # quality_group: categorize
        X_out.quality_group = X_in.quality_group.astype(self.quality_group_type).cat.codes
        
        # quantity: categorize
        X_out.quantity = X_in.quantity.astype(self.quantity_type).cat.codes
        
        # categorize non ordered columns
        for col in self.col_to_cat:
            X_out[col] = X_out[col].astype('category').cat.codes
        
        # drop columns
        X_out.drop(columns=self.col_to_drop,inplace=True)
        
        return X_out

## Import data from csv files

In [7]:
X_train_source = pd.read_csv('../Data/Training Set Values.csv',index_col='id', 
                      parse_dates=['date_recorded'])

In [9]:
y_train_source = pd.read_csv('../Data/Training Set Labels.csv', index_col='id')

In [11]:
X_test_source = pd.read_csv('../Data/Test Set Values.csv',index_col='id', 
                      parse_dates=['date_recorded'])

## Clean data

Clean X_train

In [12]:
cw = clean_water()

In [15]:
cw.fit(X_train_source)

In [24]:
X_train = cw.transform(X_train_source)

Clean y_train

In [30]:
# Label
y_train = pd.Series()
labels = ['non functional','functional needs repair', 'functional']
status_group_type = pd.CategoricalDtype(categories=labels, ordered=True)
y_train = y_train_source.status_group.astype(status_group_type).cat.codes

Clean X_test

In [32]:
X_test = cw.transform(X_test_source)

## Train model

In [35]:
model = OneVsRestClassifier(RandomForestClassifier(n_estimators=50, max_depth=50))

In [36]:
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          n_jobs=None)

## Predict labels for test data

In [38]:
y_pred = model.predict(X_test)

## Format and save predictions

In [42]:
y_format = pd.DataFrame(y_pred, index=X_test.index, columns=['status_group'])

In [44]:
y_format.status_group = y_format.status_group.replace(to_replace=[0,1,2], value=labels)

In [46]:
y_format.to_csv('../Data/PredictedLabels.csv')